<a href="https://colab.research.google.com/github/ms1515/AI-NLP/blob/main/model_pretraining/pretraining_roberta_model_with_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
%%html
<div style="background-color: yellow;">
another notebook with a slightly different use case and methodology
  <br>
  <a target="_blank" href="https://gist.github.com/aditya-malte/2d4f896f471be9c38eb4d723a710768b">
    https://gist.github.com/aditya-malte/2d4f896f471be9c38eb4d723a710768b
  </a>
</div>


# How to train a new language model from scratch using Transformers and Tokenizers

### Notebook edition (link to blogpost [link](https://huggingface.co/blog/how-to-train)). Last update May 15, 2020


Over the past few months, we made several improvements to our [`transformers`](https://github.com/huggingface/transformers) and [`tokenizers`](https://github.com/huggingface/tokenizers) libraries, with the goal of making it easier than ever to **train a new language model from scratch**.

In this post we’ll demo how to train a “small” model (84 M parameters = 6 layers, 768 hidden size, 12 attention heads) – that’s the same number of layers & heads as DistilBERT – on **Esperanto**. We’ll then fine-tune the model on a downstream task of part-of-speech tagging.


## 1. Find a dataset

First, let us find a security corpus dataset, that is published as part of the paper "Attack Attribution Based On Threat Intelligence Reports"

<img src="https://huggingface.co/blog/assets/01_how-to-train/oscar.png" style="margin: auto; display: block; width: 260px;">

Threat Intelligence Corpus is a public repository of reports (sorted by the attackers) related to malicious campaigns/activity/software that have been associated with vendor-defined APT (Advanced Persistent Threat) groups and/or tool-sets.




In [2]:

!wget -c https://github.com/eyalmazuz/ThreatIntelligenceCorpus


--2022-08-28 19:06:45--  https://github.com/eyalmazuz/ThreatIntelligenceCorpus
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ThreatIntelligenceCorpus’

ThreatIntelligenceC     [ <=>                ] 162.37K  --.-KB/s    in 0.02s   

2022-08-28 19:06:45 (6.91 MB/s) - ‘ThreatIntelligenceCorpus’ saved [166265]



## 2. Train a tokenizer

We choose to train a byte-level Byte-pair encoding tokenizer (the same as GPT-2), with the same special tokens as RoBERTa. Let’s arbitrarily pick its size to be 52,000.

We recommend training a byte-level BPE (rather than let’s say, a WordPiece tokenizer like BERT) because it will start building its vocabulary from an alphabet of single bytes, so all words will be decomposable into tokens (no more `<unk>` tokens!).


In [3]:
# We won't need TensorFlow here
# Install `transformers` from master
# !pip install git+https://github.com/huggingface/transformers
!pip install datasets transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.2 MB/s 
     |████████████████████████████████| 4.7 MB 76.4 MB/s 
     |████████████████████████████████| 115 kB 80.1 MB/s 
     |████████████████████████████████| 212 kB 79.3 MB/s 
     |████████████████████████████████| 120 kB 86.7 MB/s 
     |████████████████████████████████| 127 kB 93.9 MB/s 
     |████████████████████████████████| 6.6 MB 65.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
tokenizers                    0.12.1
transformers                  4.21.2


In [4]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 10.1 ms, sys: 3.03 ms, total: 13.1 ms
Wall time: 12.3 ms


Now let's save files to disk

In [5]:
!mkdir securityBERT
tokenizer.save_model("securityBERT")

['securityBERT/vocab.json', 'securityBERT/merges.txt']

🔥🔥 Wow, that was fast! ⚡️🔥

We now have both a `vocab.json`, which is a list of the most frequent tokens ranked by frequency, and a `merges.txt` list of merges.


What is great is that our tokenizer is optimized for the security domain. Compared to a generic tokenizer trained for text. We also represent sequences in a more efficient manner. Here on this corpus, the average length of encoded sequences is ~30% smaller as when using the pretrained GPT-2 tokenizer.

Here’s  how you can use it in `tokenizers`, including handling the RoBERTa special tokens – of course, you’ll also be able to use it directly from `transformers`.


In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./securityBERT/vocab.json",
    "./securityBERT/merges.txt",
)

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [8]:
tokenizer.encode("Security of infratructure")

Encoding(num_tokens=27, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [9]:
tokenizer.encode("server was hacked").tokens

['<s>',
 's',
 'e',
 'r',
 'v',
 'e',
 'r',
 'Ġ',
 'w',
 'a',
 's',
 'Ġ',
 'h',
 'a',
 'c',
 'k',
 'e',
 'd',
 '</s>']

## 3. Train a language model from scratch

**Update:** This section follows along the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/legacy/run_language_modeling.py) script, using our new [`Trainer`](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py) directly. Feel free to pick the approach you like best.

> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details).

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [10]:
# Check that we have a GPU
!nvidia-smi

Sun Aug 28 19:06:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

### We'll define the following config for the model

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

Now let's re-create our tokenizer in transformers

In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./securityBERT", max_len=512)

Finally let's initialize our model.

**Important:**

As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

In [14]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [15]:
model.num_parameters()
# => 84 million parameters

83504416

### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [16]:
%%time
from transformers import LineByLineTextDataset

url = 'https://github.com/eyalmazuz/ThreatIntelligenceCorpus/archive/refs/heads/master.zip' 

from datasets import load_dataset
dataset = tf.keras.utils.get_file('master.zip', url,
                                  extract=True, cache_dir='.',
                                  cache_subdir='')
# dataset = load_dataset('ThreatIntelligenceCorpus-master', 'Corpus/*', split="train")
# dataset = load_dataset("text", data_files={"train": 'ThreatIntelligenceCorpus-master/Corpus/*'}, split="train")
# dataset = load_dataset("text", data_files=['/content/ThreatIntelligenceCorpus-master/Corpus/00002822.txt', '/content/ThreatIntelligenceCorpus-master/Corpus/101718.txt', '/content/ThreatIntelligenceCorpus-master/Corpus/101718.txt'], split="train")
dataset = load_dataset("text", data_dir='ThreatIntelligenceCorpus-master/Corpus', split='train')

# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="./ThreatIntelligenceCorpus-master/corpus/",
#     block_size=128,
# )


print(dataset[0:100])

41304064/Unknown - 4s 0us/step

Resolving data files:   0%|          | 0/29 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-433958c923a1a082/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.
{'text': ['Pulling the Brake on the Magnitude EK Train', '', 'This blog goes into detail on recent work that Unit 42 has done to identify malicious sites associated with the Magnitude Exploit Kit (EK). It details the investigation process involved in identifying the algorithm used to generate domains used by the Magnitude EK. Defenders can use the provided data to identify possible domains that may be associated with the Magnitude EK before they’re used and block them pre-emptively and so block Magnitude EK attacks before they happen.', 'Initial Assessment', 'While hunting for new malware in Palo Alto Networks AutoFocus, I stumbled across some Adobe Flash files being used in what appeared to be an active exploit kit to which some users were being redirected. As I started to 

Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./securityBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


### Start training

In [19]:
# %%time
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: text. If text are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 0
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 7
Using deprecated `--per_gpu_train_batch_size` argument which wil

IndexError: ignored

#### 🎉 Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model("./securityBERT")

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./securityBERT",
    tokenizer="./securityBERT"
)

In [ ]:
# The sun <mask>.
# =>

fill_mask("La suno <mask>.")

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [ ]:
fill_mask("Jen la komenco de bela <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

## 5. Share your model 🎉

Finally, when you have a nice model, please think about sharing it with the community:

- upload your model using the CLI: `transformers-cli upload`
- write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
    - a model description,
    - training params (dataset, preprocessing, hyperparameters), 
    - evaluation results,
    - intended uses & limitations
    - whatever else is helpful! 🤓

### **TADA!**

➡️ Your model has a page on http://huggingface.co/models and everyone can load it using `AutoModel.from_pretrained("username/model_name")`.

[![tb](https://huggingface.co/blog/assets/01_how-to-train/model_page.png)](https://huggingface.co/julien-c/EsperBERTo-small)


If you want to take a look at models in different languages, check https://huggingface.co/models

[![all models](https://huggingface.co/front/thumbnails/models.png)](https://huggingface.co/models)
